In [167]:
%run Predictions.ipynb

2024-08-16 02:36:12.421390
2004-08-16 02:36:12.421390


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


           Date  ^GSPC_change  ^GSPC_lag        Close        Price
0    2004-08-16     14.539917        NaN  1079.339966  1064.800049
1    2004-08-17      2.369995  14.539917  1081.709961  1079.339966
2    2004-08-18     13.460083   2.369995  1095.170044  1081.709961
3    2004-08-19     -3.940063  13.460083  1091.229980  1095.170044
4    2004-08-20      7.119995  -3.940063  1098.349976  1091.229980
...         ...           ...        ...          ...          ...
5030 2024-08-09     37.219727  62.090332  5344.160156  5314.660156
5031 2024-08-12     25.100098  37.219727  5344.390137  5351.879883
5032 2024-08-13     65.379883  25.100098  5434.430176  5376.979980
5033 2024-08-14     58.770020  65.379883  5455.209961  5442.359863
5034 2024-08-15           NaN  58.770020  5542.339844  5501.129883

[5035 rows x 5 columns]
           Date  ^GSPC_change  ^GSPC_lag        Close        Price  \
0    2004-08-16     14.539917        NaN  1079.339966  1064.800049   
1    2004-08-17      2.369995  

In [168]:
import numpy as np

### Prepare entire dataset for calculating profits and backtesting

In [169]:
scaler = StandardScaler()
X_scaled_temp = scaler.fit_transform(input_df)
X_scaled = pd.DataFrame(X_scaled_temp, columns = input_df.columns)

In [170]:
X_seq, y_seq = create_sequences(input_df, output_df)
print(X_seq.shape)
print(y_seq.shape)

torch.Size([4248, 10, 9])
torch.Size([4248, 1])


### Final Model Predictions on entire Dataset

In [171]:
dataset = TensorDataset(X_seq, y_seq)

final_loader = DataLoader(dataset, shuffle=False)

In [172]:
model.eval()

predictions = []

with torch.no_grad():
    test_mse = []
    test_mape = []
    for X_batch, y_batch in final_loader:
        test_predictions = model(X_batch)
        mse = criterion(test_predictions, y_batch)
        mape = get_mape(y_batch, test_predictions)
        predictions.append(test_predictions)
        test_mse.append(mse.item())
        test_mape.append(mape)
    
    average_mse_loss = sum(test_mse) / len(test_mse)
    average_mape_loss = sum(test_mape) / len(test_mape)
    print(f'Average MSE: {average_mse_loss:.2f}, Average MAPE: {average_mape_loss:.2f}')

predictions_tensors = torch.cat(predictions, dim=0)
predictions_list = predictions_tensors.flatten().tolist()

Average MSE: 1145.21, Average MAPE: 533834.56


In [173]:
print(len(predictions_list))
print(predictions_list)

4248
[-6.83387565612793, -11.471919059753418, -31.85464096069336, -28.611440658569336, -22.31786346435547, -36.79466247558594, 15.383904457092285, -36.63189697265625, 5.270284175872803, 3.486008882522583, 12.51485824584961, 10.829547882080078, 23.389781951904297, 21.616453170776367, 21.340316772460938, 32.133872985839844, 35.181400299072266, 22.576475143432617, 19.851764678955078, 6.651032447814941, 6.474487781524658, 6.574248313903809, 3.3645503520965576, -3.469712972640991, -4.395862579345703, 2.537388563156128, 14.221861839294434, 21.66391372680664, 12.870085716247559, 16.282596588134766, 24.608644485473633, 27.464174270629883, 15.533135414123535, 9.860438346862793, 8.822403907775879, 8.019363403320312, 2.151463031768799, -3.163367986679077, -5.389151096343994, 6.560029029846191, 13.868884086608887, 15.395529747009277, 9.433167457580566, -4.8043413162231445, -9.383885383605957, -11.220216751098633, -27.72791290283203, -19.967914581298828, -31.08003044128418, -34.321266174316406, 11.

### Calculate Profits

In [174]:
eval_df = final_df[['Date', f'{target_stock}_change', 'Price', 'Close']]
eval_df_top10 = eval_df.iloc[:10]
eval_df_bot10 = eval_df.iloc[10:].reset_index(drop=True)
eval_df_top10['Predicted_Gain'] = np.nan
eval_df_bot10['Predicted_Gain'] = pd.DataFrame(predictions_list, columns=['Predicted_Gain'])
eval_df = pd.concat([eval_df_top10, eval_df_bot10])
print(eval_df)

           Date  ^GSPC_change        Price        Close  Predicted_Gain
0    2007-09-17     -7.609985  1484.239990  1476.650024             NaN
1    2007-09-18     43.119995  1476.630005  1519.780029             NaN
2    2007-09-19      8.939941  1519.750000  1529.030029             NaN
3    2007-09-20     -9.939941  1528.689941  1518.750000             NaN
4    2007-09-21      7.000000  1518.750000  1525.750000             NaN
...         ...           ...          ...          ...             ...
4243 2024-08-09     37.219727  5314.660156  5344.160156       29.725967
4244 2024-08-12     25.100098  5351.879883  5344.390137       31.302982
4245 2024-08-13     65.379883  5376.979980  5434.430176       32.961098
4246 2024-08-14     58.770020  5442.359863  5455.209961       28.440325
4247 2024-08-15     58.770020  5501.129883  5542.339844       20.611650

[4258 rows x 5 columns]


C:\Users\chuac\AppData\Local\Temp\ipykernel_34100\2018340844.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_df_top10['Predicted_Gain'] = np.nan


In [175]:
right_direction = 0
total = 0
for _, row in eval_df.iterrows():
    change = row[f'{target_stock}_change']
    predict = row['Predicted_Gain']
    if (change < 0 and predict <0) or (change > 0 and predict > 0):
        right_direction += 1
    total += 1
print(right_direction/total*100) 

51.92578675434476


In [176]:
# Utilise Bollinger Bands to gauge trend and decide on whether to long or short
def calc_profit(data):
    # Applying Bollinger Bands with Exponential Moving Average
    window = 20  
    data['EMA'] = data['Close'].ewm(span=window, adjust=True).mean()
    data['STD'] = data['Close'].rolling(window=window).std()
    data['Upper_Band'] = data['EMA'] + (2 * data['STD'])
    data['Lower_Band'] = data['EMA'] - (2 * data['STD'])
    data.dropna(inplace=True, ignore_index=True)
    
    # Decide on whether to short or long the stock
    orders = []
    for _, row in data.iterrows():
        if row['Price'] > row['Upper_Band']:
            orders.append(-1)
        elif row['Price'] < row['Lower_Band']:
            orders.append(1)
        elif row['Predicted_Gain'] > 0:
            orders.append(1)
        elif row['Predicted_Gain'] < 0:
            orders.append(-1) 
    
    data['Order'] = pd.DataFrame(orders, columns=['Order'])
     
    data['Profit Per Trade'] = data[f'{target_stock}_change'] * data['Order']
    data['Cumulative Profit'] = data['Profit Per Trade'].cumsum()
    data['Hold'] = data[f'{target_stock}_change'].cumsum()
    return data

In [177]:
calc_profit(eval_df)

,Date,^GSPC_change,Price,Close,Predicted_Gain,EMA,STD,Upper_Band,Lower_Band,Order,Profit Per Trade,Cumulative Profit,Hold
0,2007-10-12,6.839966,1555.410034,1561.800049,3.486009,1545.733906,21.438821,1588.611549,1502.856264,1,6.839966,6.839966,6.839966
1,2007-10-15,-14.439941,1562.250000,1548.709961,12.514858,1546.056813,16.420107,1578.897026,1513.216599,1,-14.439941,-7.599976,-7.599976
2,2007-10-16,-3.370117,1547.810059,1538.530029,10.829548,1545.250835,15.754899,1576.760633,1513.741037,1,-3.370117,-10.970093,-10.970093
3,2007-10-17,-5.149902,1544.439941,1541.239990,23.389782,1544.826376,15.523216,1575.872808,1513.779943,1,-5.149902,-16.119995,-16.119995
4,2007-10-18,0.709961,1539.290039,1540.079956,21.616453,1544.329335,14.614559,1573.558454,1515.100217,1,0.709961,-15.410034,-15.410034
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4234,2024-08-09,37.219727,5314.660156,5344.160156,29.725967,5397.493226,137.330737,5672.154700,5122.831753,1,37.219727,5180.926270,3796.469849
4235,2024-08-12,25.100098,5351.879883,5344.390137,31.302982,5392.435789,131.416930,5655.269649,5129.601929,1,25.100098,5206.026367,3821.569946
4236,2024-08-13,65.379883,5376.979980,5434.430176,32.961098,5396.435254,118.821182,5634.077618,5158.792891,1,65.379883,5271.406250,3886.949829
4237,2024-08-14,58.770020,5442.359863,5455.209961,28.440325,5402.032846,112.231220,5626.495285,5177.570406,1,58.770020,5330.176270,3945.719849
